# Converting Data

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/AustralianCancerDataNetwork/pydicer/blob/main/examples/ConvertingData.ipynb)

In this example, the preprocessing and conversion of DICOM data is demonstrated. These are
essential first steps before data can be analysed using PyDicer.

In [ ]:
try:
    from pydicer import PyDicer
except ImportError:
    !pip install pydicer
    from pydicer import PyDicer

from pathlib import Path

from pydicer.input.test import TestInput

## Setup PyDicer

As in the `Getting Started` example, we must first define a working directory for our dataset. We
also create a `PyDicer` object.

In [ ]:
directory = Path("./working")
pydicer = PyDicer(directory)

## Fetch some data

A TestInput class is provided in pydicer to download some sample data to work with. Several other
input classes exist if you'd like to retrieve DICOM data for conversion from somewhere else. See 
the [docs for information](https://australiancancerdatanetwork.github.io/pydicer/html/input.html)
on how the PyDicer input classes work.

Most commonly, if you have DICOM files stored within a folder on your file system you can simply
pass the path to your DICOM directory to the `pydicer.add_input()` function.

In [ ]:
dicom_directory = directory.joinpath("dicom")
test_input = TestInput(dicom_directory)
test_input.fetch_data()

# Add the input DICOM location to the pydicer object
pydicer.add_input(dicom_directory)

## Preprocess

With some DICOM data ready to work with, we must first use the PyDicer `preprocess` module. This
module will crawl over all DICOM data available and will index all information required for
conversion of the data.

In [ ]:
pydicer.preprocess()

### Inspect Preprocessed Data

Here we load the data that was indexed during preprocessing and output the first rows. This data
will be used by the following step of data conversion.

In [ ]:
df_preprocessed = pydicer.read_preprocessed_data()
df_preprocessed.head()

## Convert Data

With the DICOM data having been indexed during preprocessing, we are now ready to convert this data
into NIfTI format which will be stored within the PyDicer standard directory structure.

Running the following cell will begin the conversion process. While this cell is running, take a
look inside the `working/data` directory to see how the converted data is being stored.

Notice the `converted.csv` file stored for each patient. This tracks each converted data object.
This will be loaded as a Pandas DataFrame for use throughout PyDicer.


In [ ]:
pydicer.convert.convert()

### Load Converted DataFrame

Once data is converted, we can load a Pandas DataFrame which contains a description of each object
converted.

The most useful columns in the DataFrame for working with this data in PyDicer are:
- `hashed_uid`: This is a 6 character hexidecimal hash of the associated DICOM SeriesInstanceUID.
  PyDicer refers to objects using this hashed identifier for a more consice representation.
- `modality`: The modality of the data object.
- `patient_id`: The ID of the patient this data object belongs to.
- `path`: The path within the working directory where files for this data object are stored.

In [ ]:
df = pydicer.read_converted_data()
df

## Data Quarantine

If anything goes wrong while converting a DICOM object during either the preprocess step or the
conversion step, the problematic DICOM data will be copied to the `working/quarantine` directory.

It's a good idea to regularly check your quarantine directory to ensure that no critical data
objects are being quarantine. If so you may want to consider rectifying the issue and running the
preprocess and conversion steps again.

As can be seen by running the cell below, there were several DICOM objects moved to the quarantine
during for our test dataset. This was due to there being multiple slices at the same location with
differing pixel data in one CT image series.

In [ ]:
df_quarantine = pydicer.read_quarantined_data()
df_quarantine